# EAI Final Project - Désirée Bellan

Training code for the final project of the course Elective in Artificial Intelligence, held by prof. Christian Napoli and followed during the year 2020/2021

## Import Libraries

In [ ]:
!pip install pytorch-lightning==2.0.1
!pip install torch_geometric
!pip install torch-cluster==1.6.1 -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

In [ ]:
# import libraries
from torchvision import transforms as T

from sklearn.metrics import confusion_matrix, classification_report
from typing import Callable, Optional, List, Union, Dict, Tuple
from torch_geometric.utils import to_dense_adj
from torchvision import datasets, models
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_cluster import knn_graph
from torch.optim import Adam
from google.colab import drive

import torch_geometric.nn as gnn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pytorch_lightning as pl
import torch.nn as nn
import seaborn as sns
import pandas as pd
import numpy as np

import torchmetrics
import torch_geometric
import PIL
import torch
import dlib
import math

## Path to Directories (Google Drive directories used during training)

In [ ]:
# directories
drive.mount('/content/drive', force_remount = True)
dir_='/content/drive/MyDrive/uni/EAI/'
data_path = dir_ + 'data/'
model_path = dir_ + 'model/'

## Metrics

Metric utils to create plots and graphs with pytorch lightning

In [ ]:
def tocpu(list):
  return [elem.cpu().detach() for elem in list]

class Metrics(pl.Callback):
  def __init__(self, name:str, plot = True):

        self.name = name
        self.plot = plot

        self.collection = {'train_loss': [], 'train_acc': [], 'train_f1': [], 
                        'dev_loss': [], 'dev_acc': [], 'dev_f1': []}
        self.preds = torch.tensor([]).to(torch_device)
        self.targets = torch.tensor([]).to(torch_device)

        self.val_epoch = {'loss': [], 'acc': [], 'f1': []}

  def on_train_epoch_end(self, trainer, pl_module):
        self.collection['train_loss'].append(trainer.callback_metrics['train_loss'].tolist())
        self.collection['train_acc'].append(trainer.callback_metrics['train_acc'].tolist())
        self.collection['train_f1'].append(trainer.callback_metrics['train_f1'].tolist())
        
  def on_validation_epoch_end(self, trainer, pl_module):
        self.val_epoch['loss'].append(trainer.callback_metrics['valid_loss'].tolist())
        self.val_epoch['acc'].append(trainer.callback_metrics['valid_acc'].tolist())
        self.val_epoch['f1'].append(trainer.callback_metrics['valid_f1'].tolist())
    
  def on_validation_end(self, trainer, pl_module):
        self.collection['dev_loss'].append(sum(self.val_epoch['loss'])/len(self.val_epoch['loss']))
        self.collection['dev_acc'].append(sum(self.val_epoch['acc'])/len(self.val_epoch['acc']))
        self.collection['dev_f1'].append(sum(self.val_epoch['f1'])/len(self.val_epoch['f1']))
        self.val_epoch['loss'] = []
        self.val_epoch['acc'] = []
        self.val_epoch['f1'] = []

  def on_test_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
      self.preds = torch.cat([self.preds, outputs['preds']])
      self.targets = torch.cat([self.targets, outputs['targets']])
    
  def on_test_end(self, trainer, pl_module):
      num_classes = pl_module.hparams['output-channels']
      if self.plot:
        # PLOT CURVES
        fig, axs = plt.subplots(1,3, figsize = (9, 3))
        x = range(len(self.collection['train_loss']))
        axs[0].set_title('Loss')
        axs[0].plot(x, self.collection['train_loss'], label = 'train')
        axs[0].plot(x,self.collection['dev_loss'][1:len(self.collection['train_loss'])+1], label = 'valid')
        #axs[0].set_ylim(bottom = -200, top = 100)
        axs[1].set_title('Accuracy')
        axs[1].plot(x, self.collection['train_acc'], label = 'train')
        axs[1].plot(x, self.collection['dev_acc'][1:len(self.collection['train_loss'])+1], label = 'valid')
        axs[2].set_title('F1 Score')
        axs[2].plot(x, self.collection['train_f1'], label = 'train')
        axs[2].plot(x, self.collection['dev_f1'][1:len(self.collection['train_loss'])+1], label = 'valid')

        handles, labels = [a for a in axs[0].get_legend_handles_labels()]
        fig.legend(handles, labels, loc='upper right')

        plt.show()
        plt.savefig('{}_plot.png'.format(self.name))

      # PLOT CONFUSION MATRIX
      task = 'multiclass'
      num_classes = 2 
      ConfusionMatrix = torchmetrics.ConfusionMatrix(task, num_classes=num_classes).to(torch_device)
      plt.figure(figsize = (8,8))
      cfm = ConfusionMatrix(self.preds.float(), self.targets).cpu().detach().numpy()
      for i in range(num_classes):
          cfm[:,i] = cfm[:,i]/np.sum(cfm[:,i])*100 if np.sum(cfm[:,i])!= 0 else cfm[:,i]
      class_names = ['no-syndrom', 'syndrom'] if num_classes == 2 else list(range(1, num_classes + 1))
      dataframe = pd.DataFrame(cfm, index=class_names, columns=class_names)
      # Create heatmap
      sns.heatmap(dataframe, annot = True, cmap="YlGnBu", xticklabels=2) 
      plt.title("Confusion Matrix")
      
      plt.ylabel("True Class"), 
      plt.xlabel("Predicted Class")
      plt.show()
      plt.savefig('{}_cfm.png'.format(self.name))

      del dataframe
      del cfm

      self.preds = (self.preds >= 0.5).int()
      print(classification_report(tocpu(self.targets), tocpu(self.preds)))

  def on_save_checkpoint(self, trainer, pl_module, checkpoint) -> None:
        checkpoint['metrics'] = {'collection': self.collection}
        checkpoint['current_epoch'] = trainer.current_epoch
        
  def load(self, ckpt_path) -> None:
        checkpoint = torch.load(ckpt_path, map_location=torch_device) 
        self.collection = checkpoint['metrics']['collection']

torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpus = 1 if torch_device == torch.device('cuda') else  0

## Datasets

### Landmark Identification Utils

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(model_path + '/dlib/shape_predictor_68_face_landmarks.dat')

### Graph Dataset

In [ ]:
class faceGraphDataset(datasets.ImageFolder):
    def __init__(self, root):
      super().__init__(root)
      self.data = []
      for i, (image_path, target) in enumerate(self.samples):
        self.create_graph(image_path, target, i)

    def create_graph(self, image_path:str, target:int, batch_idx:int):
      # Load face image
      img = dlib.load_rgb_image(image_path)
      # Detect face
      faces = detector(img, 1)
      if len(faces) > 0:
        # Detect facial landmarks
        landmarks = predictor(img, faces[0])
        landmarks = [[landmarks.part(i).x, landmarks.part(i).y] for i in range(68)]
        # Define nodes
        nodes = torch.tensor(landmarks, dtype=torch.float)
        batch = torch.tensor([batch_idx for _ in range(nodes.shape[0])])
        # Define edges
        edges = knn_graph(nodes, k=6, batch = batch) # connect each node to its 6 nearest neighbors
        # Define weights
        distances = torch.norm(x[edges[0]] - x[edges[1]], p=2, dim=1)
        # Assign features
        features = torch.zeros((68, 3, 32, 32)) # create a tensor to hold features
        for i, landmark in enumerate(landmarks):
            x, y = landmark
            x += -min(0, x-16) - max(max(img.shape[0], x+16) - img.shape[0],0)
            y += -min(0, y-16) - max(max(img.shape[1], y+16) - img.shape[1],0)
            feature = img[x-16:x+16, y-16:y+16] # crop a 32x32 region around the landmark point
            features[i] = torch.tensor(feature, dtype=torch.float).permute(2, 0, 1) # convert to tensor and permute dimensions to match PyTorch convention
        # Create adjacency matrix
        adj = to_dense_adj(edges)
        # Create PyTorch Geometric Data object
        self.data.append((Data(x=features, edge_index=edges, edge_attr=adj, edge_weights=distances), target))

    def __getitem__(self, index):
      return self.data[index]
    def __len__(self):
      return len(self.data)

### VGG Dataset

In [ ]:
class faceDataset(datasets.ImageFolder):

  def shape_to_np(self, shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
      coords[i] = (shape.part(i).x, shape.part(i).y)
    # return the list of (x, y)-coordinates
    return coords

  def find_features(self, path, img_to_crop):
    # find landmarks
    img = dlib.load_rgb_image(path)
    face = detector(img, 1)
    features = []
    if len(face)>0:
      face = face[0]
      shape = predictor(img, face)
      shape = self.shape_to_np(shape)
      if landmarks == 1:
        features = [img_to_crop.crop((face.left(), face.top(), face.right(), face.bottom()))]
      # case divided into three parts : eyes, nose, mouth
      if landmarks == 3:
        eyes = img_to_crop.crop((face.left(), face.top(), face.right(), shape[29][1]))
        nose = img_to_crop.crop((face.left(), shape[21][1], face.right(), shape[50][1]))
        mouth = img_to_crop.crop((face.left(), shape[33][1], face.right(), face.bottom()))
        features = [eyes, nose, mouth]
      # case divided into four parts : left eye, right eye, nose, mouth
      elif landmarks == 4:
        leye = img_to_crop.crop((shape[17][0],shape[20][1],shape[27][0],shape[29][1]))
        reye = img_to_crop.crop((shape[27][0],shape[24][1],shape[26][0],shape[29][1]))
        nose = img_to_crop.crop((shape[40][0],shape[27][1],shape[47][0],shape[51][1]))
        mouth = img_to_crop.crop((shape[41][0],shape[33][1],shape[46][0],shape[5][1]))
        features = [leye, reye, nose, mouth]
    else:
      w, h = img_to_crop.size
      if landmarks ==  1:
        features = [img_to_crop]
      if landmarks == 3:
        eyes = img_to_crop.crop((0, 0, w, h//3))
        nose = img_to_crop.crop((0, h//3, w, h*2//3))
        mouth = img_to_crop.crop((0, h*2//3, w, h))
        features = [eyes, nose, mouth]
      elif landmarks == 4:
        leye = img_to_crop.crop((0,0,w//2,h//3))
        reye = img_to_crop.crop((w//2,0,w,h//3))
        nose = img_to_crop.crop((0,h//3,w,2*h//3))
        mouth = img_to_crop.crop((0,2*h//3,w,h))
        features = [leye, reye, nose, mouth]
    return features

  def __getitem__(self, index:int):
    path, target = self.samples[index]
    sample = self.loader(path)  
    # divide image into landmarks
    faceFeatures= self.find_features(path, sample)
    input = {'input' : sample}
    for i, feature in enumerate(faceFeatures):
      input[i] = feature
    for i in input:
      if self.transform is not None:
        input[i] = self.transform(input[i])
    if self.target_transform is not None:
      target = self.target_transform(target)
    return input, target

## Dataloaders

### Graph Dataloader

In [ ]:
class graphDataloader(pl.LightningDataModule):
  def __init__(self, root:str, batch_size:int, gan:bool = False):
    super().__init__()
    self.root = root
    self.batch_size = batch_size
    self.train_path = root + 'train' if not gan else root + 'train_augmented'
    self.dev_path = root + 'dev' if not gan else root + 'dev_augmented'
    self.test_path = root + 'test' if not gan else root + 'test_augmented'
  def setup(self, stage: Optional[str] = None) -> None:
      if stage == "fit":
        self.train_dataset = faceGraphDataset(self.train_path)
        try: self.validation_dataset = faceGraphDataset(self.dev_path)
        except: self.validation_dataset = faceGraphDataset(self.test_path)
      if stage == 'validate':
        try : self.validation_dataset = faceGraphDataset(self.dev_path)
        except : self.validation_dataset = faceGraphDataset(self.test_path)
      if stage == 'test':
        self.test_dataset = faceGraphDataset(self.test_path)
  def train_dataloader(self, *args, **kwargs) -> DataLoader:
        return torch_geometric.loader.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return torch_geometric.loader.DataLoader(self.validation_dataset, batch_size=self.batch_size, shuffle=True)

  def test_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return torch_geometric.loader.DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

### VGG Dataloader

In [ ]:
class dataloader(pl.LightningDataModule):
  def __init__(self, root:str, transform:T, batch_size:int, augment:bool = True, gan:bool = False):
    super().__init__()
    self.batch_size = batch_size
    self.train_transform = self.test_transform = transform
    if augment : 
      self.train_transform = T.Compose([
          T.RandomHorizontalFlip(), 
          T.RandomAffine(degrees = (-5,5), translate = (0.05, 0.05), shear = 5*math.pi/180),
          T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
          T.RandomEqualize(),
          T.GaussianBlur(3),
          self.train_transform,
          T.RandomErasing()])
    self.train_path = root + 'train' if not gan else root + 'train_augmented'
    self.dev_path = root + 'dev' if not gan else root + 'dev_augmented'
    self.test_path = root + 'test' if not gan else root + 'test_augmented'
  def setup(self, stage: Optional[str] = None) -> None:
      if stage == "fit":
        self.train_dataset = faceDataset(self.train_path, transform = self.train_transform)
        try: self.validation_dataset = faceDataset(self.dev_path, transform = self.test_transform)
        except: self.validation_dataset = faceDataset(self.test_path, transform = self.test_transform)
      if stage == 'validate':
        try : self.validation_dataset = faceDataset(self.dev_path, transform = self.test_transform)
        except : self.validation_dataset = faceDataset(self.test_path, transform = self.test_transform)
      if stage == 'test':
        self.test_dataset = faceDataset(self.test_path, transform = self.test_transform)
  def train_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.validation_dataset, batch_size=self.batch_size, shuffle=True)

  def test_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

## VGG Model

In [ ]:
from drive.MyDrive.uni.EAI.data.vgg_face import vgg_face_dag

class classification(pl.LightningModule):
  def __init__(self, hparams):
    super(classification, self).__init__()
    self.hparams.update(hparams)
    self.hparams['VGG_FACE_WEIGHTS'] = data_path + 'vgg_face_dag.pth'
    self.save_hyperparameters()
    self.model = vgg_face_dag(self.hparams['VGG_FACE_WEIGHTS'])
    self.model.fc8 = nn.Linear(4096, self.hparams['output-channels'], bias = True)
    self.loss = nn.CrossEntropyLoss(weight = torch.tensor([self.hparams['weight'], 1.]))
    task = 'multiclass'
    self.val_f1 = torchmetrics.F1Score(task = task, num_classes = self.hparams['output-channels'], average = 'macro')
    self.accuracy = torchmetrics.Accuracy(task = task, num_classes = self.hparams['output-channels'], average = 'macro')
  def forward(self, x, last_layer = True):
    return F.softmax(self.model(x, last_layer), dim = -1)

  def training_step(self, batch, batch_idx):
    inputs, labels = batch
    feature = inputs[self.hparams['feature']]
    logits = self.forward(feature)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    #print(prediction)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    self.log('train_loss', loss.item(), prog_bar=True)
    self.log('train_f1', sample_f1*100)
    self.log('train_acc', sample_accuracy*100)

    return loss

  def validation_step(self, batch, batch_idx):
    inputs, labels = batch
    feature = inputs[self.hparams['feature']]
    logits = self.forward(feature)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)

    self.log('valid_f1', sample_f1*100, prog_bar=True)
    self.log('valid_acc', sample_accuracy*100, prog_bar=True)
    self.log('valid_loss', loss)

  def test_step(self, batch, batch_idx):
    inputs, labels = tuple(batch)
    feature = inputs[self.hparams['feature']]
    logits = self.forward(feature)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)

    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    return {'preds': predictions, 'targets': labels}

  def configure_optimizers(self):
    return Adam(self.parameters(), lr = self.hparams['lr'], weight_decay = 1e-2)

## N-Feature Combination Classifier

In [ ]:
class ClassificationUsingFeatures(pl.LightningModule):
  def __init__(self, hparams:dict, features:List[str]):
    super(ClassificationUsingFeatures, self).__init__()
    self.hparams.update(hparams)
    self.save_hyperparameters()
    self.features = nn.ModuleList([classification.load_from_checkpoint(path) for path in features])
    for module in self.features:
      for param in module.parameters():
        param.requires_grad = False
    input_dim = 2*len(self.features)
    self.classifier = nn.Sequential(
        nn.Linear(input_dim,input_dim*8),
        nn.Dropout(self.hparams['dropout']),
        nn.ReLU(),
        nn.Linear(input_dim*8,input_dim*4),
        nn.Dropout(self.hparams['dropout']),
        nn.ReLU(),
        nn.Linear(input_dim*4,2))
    self.loss = nn.CrossEntropyLoss(weight = torch.tensor([self.hparams['weight'], 1.]))
    self.val_f1 = torchmetrics.F1Score(task = "multiclass", num_classes = 2, average = 'macro')
    self.accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes = 2, average = 'macro')

  def forward(self, features):
    for i, module in enumerate(self.features):
      features[i] = module(features[i])
    x = torch.cat(features, dim=1)
    return F.softmax(self.classifier(x), dim = -1)

  def training_step(self, batch, batch_idx):
    inputs, labels = batch
    features = [image for key, image in inputs.items()]
    logits = self.forward(features)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    self.log('train_loss', loss.item(), prog_bar=True)
    self.log('train_f1', sample_f1*100)
    self.log('train_acc', sample_accuracy*100)

    return loss

  def validation_step(self, batch, batch_idx):
    inputs, labels = batch
    features = [image for key, image in inputs.items()]
    logits = self.forward(features)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)

    self.log('valid_f1', sample_f1*100, prog_bar=True)
    self.log('valid_acc', sample_accuracy*100, prog_bar=True)
    self.log('valid_loss', loss)

  def test_step(self, batch, batch_idx):
    inputs, labels = batch
    features = [image for key, image in inputs.items()]
    logits = self.forward(features)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)

    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    return {'preds': predictions, 'targets': labels}

  def configure_optimizers(self):
    return Adam(self.parameters(), lr = self.hparams['lr'], weight_decay = 1e-2)

## Graph CNNs Face Classification Model

In [ ]:
class GCNNFaceRecognition(pl.LightningModule):
  def __init__(self, hparams, num_features=68 , hidden_channels = 128, num_classes = 2):
        super(GCNNFaceRecognition, self).__init__()
        self.hparams.update(hparams)
        self.save_hyperparameters()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size = 3, padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size = 3, padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size = 3, padding = 1, stride = 2),
            nn.ReLU()
        )
        self.graph_cnn1 = gnn.GCNConv(256, hidden_channels)
        self.graph_cnn2 = gnn.GCNConv(hidden_channels, 2)
        self.fc = nn.Linear(num_features, 1)
        self.loss = nn.CrossEntropyLoss(weight = torch.tensor([self.hparams['weight'], 1.]))
        self.val_f1 = torchmetrics.F1Score(task = "multiclass", num_classes = 2, average = 'macro')
        self.accuracy = torchmetrics.Accuracy(task = "multiclass", num_classes = 2, average = 'macro')

  def forward(self, data):
        num_graphs = data.num_graphs
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_weight
        num_features = x.shape[0]//num_graphs
        x = self.conv(x).squeeze()
        x = F.relu(self.graph_cnn1(x, edge_index, edge_weight))
        x = F.relu(self.graph_cnn2(x, edge_index, edge_weight))
        x = self.fc(x.reshape(num_graphs, num_features, 2).permute(0,2,1)).squeeze()
        return x
  def training_step(self, batch, batch_idx):
    input, labels = batch
    logits = self.forward(input)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    self.log('train_loss', loss.item(), prog_bar=True)
    self.log('train_f1', sample_f1*100)
    self.log('train_acc', sample_accuracy*100)

    return loss

  def validation_step(self, batch, batch_idx):
    input, labels = batch
    logits = self.forward(input)
    loss = self.loss(logits, labels)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)

    self.log('valid_f1', sample_f1*100, prog_bar=True)
    self.log('valid_acc', sample_accuracy*100, prog_bar=True)
    self.log('valid_loss', loss)

  def test_step(self, batch, batch_idx):
    input, labels = batch
    logits = self.forward(input)
    predictions = torch.argmax(logits, dim = -1).view(-1)
    labels = labels.view(-1)
    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)

    sample_f1 = self.val_f1(predictions, labels)
    sample_accuracy = self.accuracy(predictions, labels)
 
    return {'preds': predictions, 'targets': labels}

  def configure_optimizers(self):
    return Adam(self.parameters(), lr = self.hparams['lr'], weight_decay = 1e-2)

## Training

#### Whole Face 

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 1
hparams = {
    'augment' : False,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 0,
    'optimizer': 'Adam',
    'landmarks' : 1,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])
trainer.fit(model, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, augmentation = True, gan = False, batch_size = 32
ckpt_path = model_path + 'best_checkpoints/best_whole-face_augment_95.1515.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, augmentation = False, gan = True, batch_size = 32
ckpt_path = model_path + 'best_checkpoints/best_whole-face_weight=0.3_loss=0.337_noagumented_gan.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, augmentation = False, gan = False, batch_size = 32
ckpt_path = model_path + 'best_checkpoints/best_whole-face_noaugment(97.6698).ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.2, augmentation = True, gan = True, batch_size = 32
ckpt_path = model_path + 'best_checkpoints/best_whole-face_weight=0.2_loss=0.321_gan.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# GCNNs weights = 0.2, gan = False, batch_size = 16
ckpt_path = model_path + 'best_checkpoints/best_whole-face_graph.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# GCNNs weights = 0.2, gan = True, batch_size = 32
ckpt_path = model_path + 'best_checkpoints/best_whole-face_graph_gan.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

### Eyes, Nose And Mouth Features

#### Eyes

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 3
hparams = {
    'augment' : True,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 0,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG_eyes')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = True, gan = False
ckpt_path = model_path + 'best_checkpoints/best_eyes_weights=0.3_88.3491.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = False, gan = False
ckpt_path = model_path + 'best_checkpoints/best_eyes_weights=0.3_noaugment_96.1212.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

#### Nose

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 3
hparams = {
    'augment' : True,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 1,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG_nose')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = True, gan = False
ckpt_path = model_path + 'best_checkpoints/best_nose'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = False, gan = False
ckpt_path = model_path + 'best_checkpoints/best_nose_weight=0.3_noaugment_78.9182.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

#### Mouth

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 3
hparams = {
    'augment' : True,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 2,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG_mouth')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])

In [ ]:
# VGG weights = 0.2, batch_size = 32, augment = True, gan = False
ckpt_path = model_path + 'best_checkpoints/best_mouth_weights=0.2_70-9126.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = False, gan = False
ckpt_path = model_path + 'best_checkpoints/best_mouth_weights=0.3_noaugment.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

### Forehead - Ears and Chin

#### Forehead and Ears

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 2
hparams = {
    'augment' : True,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 0,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG_forehead')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = True, gan = False
ckpt_path = model_path + 'best_checkpoints/best_forehead.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.2, batch_size = 32, augment = True, gan = True
ckpt_path = model_path + 'best_checkpoints/best_forehead_weight=0.2_loss=0.314_gan.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = False, gan = False
ckpt_path = model_path + 'best_checkpoints/best_forehead_weight=0.3_loss_3.56_noaugmentation.ckpt'
trainer = pl.Trainer()
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

#### Chin

In [ ]:
# VGG16 Model with augmentation for syndrome/no-syndrome classification pretrained over vgg-face
landmarks = 2
hparams = {
    'augment' : True,
    'max_epochs' : 500,
    'batch_size': 32,
    'input-channels':3,
    'output-channels':2,
    'feature' : 1,
    'VGG_FACE_WEIGHTS':data_path + 'vgg_face_dag.pth',
    'vgg' : 'vgg-face',
    'lr': 2e-5,
    'weight': 0.2
    }
checkpoint_dir = model_path + 'VGG'
check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='valid_loss',  
    verbose=True, 
    save_top_k = 2,  
    mode='min', 
    dirpath= checkpoint_dir,  
    filename='{epoch}-{valid_f1:.4f}')
early_stopping = pl.callbacks.EarlyStopping(
    monitor='valid_loss',  
    patience = 10,  
    verbose=True,  
    mode='min',
)
model = classification(hparams)
model.to(torch_device)
model.train()
metric = Metrics(name = 'VGG_chin')
transform = models.VGG16_Weights.IMAGENET1K_V1.transforms
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer = pl.Trainer(val_check_interval=1.0, max_epochs = hparams['max_epochs'], default_root_dir= checkpoint_dir,  callbacks = [check_point_callback, metric, early_stopping])


In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = True, gan = False
ckpt_path = model_path + 'best_checkpoints/best_chin'
trainer = pl.Trainer()
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

In [ ]:
# VGG weights = 0.3, batch_size = 32, augment = False, gan = False
ckpt_path = model_path + 'best_checkpoints/best_chin_weight=0.3_loss=0.35_noaugmented.ckpt'
trainer = pl.Trainer()
data_module = dataloader(data_path, transform, hparams['batch_size'], augment = hparams['augment'], gan = hparams['gan'])
trainer.test(model = model, ckpt_path = ckpt_path, datamodule = data_module)

## PA-GAN

In [ ]:
!pip install tensorflow-graphics-gpu --no-deps oyaml tensorflow-addons tf-slim

In [ ]:
%cd /content/drive/MyDrive/uni/EAI/model/PA-GAN

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/uni/EAI/model/PA-GAN/test.py \
--experiment_name PA-GAN_128 \
--img_dir /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/aligned/data \
--test_label_path /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/aligned/test_label.txt

In [ ]:
!python /content/drive/MyDrive/uni/EAI/model/PA-GAN/scripts/align.py \
--img_dir /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/images \
--save_dir /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/aligned \
--standard_landmark_file /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/standard_landmark_68pts.txt\
--landmark_file /content/drive/MyDrive/uni/EAI/model/PA-GAN/data/landmarks.txt